(explainer_data_collection)=
Data collection
==================

This notebook outlines our data collection strategy which consists of the following steps:

1. Finding the relevant Wikipedia pages for each discipline through [PetScan](https://petscan.wmflabs.org/).
2. Scraping each page to parse out hyperlinks to other Wikipedia pages and the text.
3. Creating a smaller and manageble subgraph from the Network.

Lastly we will also describe the steps that we have taken to preprocess our data for the purphose of later natural language processing.

In [101]:
# imports
import re
import json
import string
import random 
import requests
import pickle
import warnings
import powerlaw
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import List
from itertools import chain
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from dataclasses import dataclass

import networkx as nx
import littleballoffur

import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Finding relevant articles

To collect the relevant wikipedia pages for our project we specify the dataclass `WikiPage`. This is based on the use of the open-source software [PetScan](https://petscan.wmflabs.org/) that based on a list of wikipedia-categories yields the corresponding page-names. We furthermore specify the depth of our PetScan-query, which is a measure of how deeply nested we want our categories to be. As the list of pages grows exponentially we limit the levels of depth we set the parameter to 0, 1 and 2. The reason for not choosing one specific depth is that the group and sub-group structure of the disciplines differs which means that we get a widely different amount of pages. 

In [2]:
@dataclass(frozen=False)
class WikiPage:
    """
    Data obj that stores an article and 
    its relevant attributes
    """
    title:str
    parent:str
    depth:int
    text:str = np.nan
    edges:List = np.nan
        

def collect_pages(parents:list,
                  depth:int=0)->List[WikiPage]:
    
    """
    Finds relevant articles from petscan based on some initial query.
    See https://petscan.wmflabs.org/ for api reference.
    """
    
    pages = list()
    errors = 0
    #setup API call
    base_url = 'https://petscan.wmflabs.org/?ns%5B0%5D=1&'
    params = {'project':'wikipedia',
              'language':'en',
              'format':'json',
              'interface_language':'en',
              'depth':str(depth),
              'doit':''}
    
    #Loop over parents and get corresponding page names
    for cat in parents:
        params['categories'] = cat
        resp = requests.get(url=base_url, params=params).json()
        try: 
            for page in resp['*'][0]['a']['*']:

                #Append nodes
                pages.append(WikiPage(title=page['title'],
                                      parent=cat,
                                      depth=depth))
                
        except KeyError:
            errors+=1
    
    print(f'Petscan failed to retrieve {errors} pages in depth {depth}...')
    
    return pages

Bellow we call the function `collect_pages` and create a page list for depth 0, 1 and 2 and display the resulting counts. As can be seen Anthropology is a clear outlier because of a different group structure on wikipedia.

In [3]:
#Define initial query groups
query = ['political_science', 'economics', 'sociology', 'anthropology', 'psychology']

depths = [0,1,2]
pages = []
for d in tqdm(depths):
    pages += collect_pages(query, d)
    
#Show marginal distribution of pages
pd.DataFrame(pages).groupby('parent').count()['title']

  0%|          | 0/3 [00:00<?, ?it/s]

Petscan failed to retrieve 0 pages in depth 0...
Petscan failed to retrieve 0 pages in depth 1...
Petscan failed to retrieve 0 pages in depth 2...


parent
anthropology         17621
economics             6023
political_science     7011
psychology            8757
sociology             5895
Name: title, dtype: int64

## Collect page text and edges

In the function `collect_attributes` we use `BeautifulSoup` to scrape the html content from the wikipedia pages we've found. The key html node is the `div` with attributes `{'id':'mw-content-text'}` from which we can parse out all paragraphs and hyperlinks, disregarding section headings, tables and other irrelevant content and page attributes.

In [17]:
def collect_attributes(articles:list[WikiPage])->list[WikiPage]:
    """
    Parses the wikipedia article text and urls pointing to another wiki page.
    """
    base_url = 'https://en.wikipedia.org/wiki/'
    error_log = dict()
    for page in tqdm(pages):
        try:
            try: 
                resp = requests.get(base_url+page.title, timeout=10)
            except requests.exceptions.Timeout as e: 
                error_log[page.title] = e
            
            soup = BeautifulSoup(resp.content, 'html.parser')
            content = soup.find('div', {'id':'mw-content-text'})
            text = ''
            for paragraph in content.find_all('p'):
                text += ' ' + paragraph.text
            page.text = text
            page.edges = [ref.text for ref in content.find_all('a', href=True) 
                                               if 'wiki' in ref.get('href')]
        except Exception as e:
            #Log potential errors in collection
            error_log[page.title] = str(e)
            
    return pages, error_log

pages, error_log = collect_attributes(pages)

  0%|          | 0/45307 [00:00<?, ?it/s]

In [18]:
pages_df = pd.DataFrame(pages)

In [26]:
print(f'Amount of pages that failed to be collected: {len(error_log.keys())}')

Amount of pages that failed to be collected: 135


As only 135 out of 45307 pages are missing we do not consider this a problem of substantiel value, why we keep out dataset the way it is.

## Subsetting a smaler network

Because of the large size of the network, we deem it necessary to create a smaller subgraph that is more manageble. To do this we initially perform three operations:

- (1) **Restrict anthropology pages to depth = 1**: 
Due to the inherent strcuture of Wikipedia we can not expect all of the categories to have an equal amount of pages. 
Some categories might be defined more loosely and therefore span broader, whereas other categories might be larger as more people are participates in activities related to the given discpline and therefore more prone to write a Wikipedia page about. No matter the reason, anthropology pages are widely overrepresentative in our dataset, why we decide to restrict the anthropology pages to `depth = 1`.
- (2) **Remove Duplicates**: 
Some pages occur in two categories e.g. the page *elitism* occur both in political science and anthropology category. As it becomes ambigious to which discipline the page belong we decide to remove them. In cases where some pages occurs several times within the same category (as they are collected at different depths) we collapse them to one observation.
- (3) **Remove edges to pages that did not collect**: 
Lastly we remove all edges from the pages we have collected to pages not in our sample. 

In [165]:
pages_df = pd.read_pickle("full_data.pickle")

In [168]:
def remove_anthro(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove pages in the anthropology category of depth 2
    """
    df = df.loc[~((df["depth"] == 2) & (df["parent"] == "anthropology"))]
    return df

def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove inter-category page-duplicates and collapse intra-category duplicates
    """
    nodes_to_remove = [node for node in tqdm(set(df[df.duplicated("title")]["title"])) if
                      len(set(df[df["title"] == node]["parent"])) > 1]
    
    df = df[~df['title'].isin(nodes_to_remove)]
    df = df.drop_duplicates(subset="title", keep="first")
    return df
    
def uniform_page_and_edge_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Uniform the spelling and format of string in the title and edges column
    """
    df['title'] = df['title'].str.lower()
    df['edges'] = df['edges'].apply(lambda x: [re.sub(' ', '_', l.strip().lower()) for l in x])
    return df

def remove_edges_not_in_nodelist(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove all edges not in the title column from the lists of edges
    """
    tqdm.pandas()
    nodes = df["title"].tolist()
    df['edges'] = df['edges'].progress_apply(lambda x: [e for e in x if e in nodes])
    return df

# Calling the functions
pd.options.mode.chained_assignment = None  # Hide Pandas Warnings
df = remove_duplicates(pages_df)
df = remove_anthro(df)
df = uniform_page_and_edge_names(df)
df = remove_edges_not_in_nodelist(df)
df = df.reset_index()

  0%|          | 0/8004 [00:00<?, ?it/s]

  0%|          | 0/20061 [00:00<?, ?it/s]

To further subset our data we remove self-loops and model a undirected network from which we extract the giant connected component. We then try to sample a respresntative supgraph containing $\frac{1}{4}$ of the giant connected component's nodes using a Metropolis algoritm as stated in {cite}`Hbler2008` and implemented in the function `.MetropolisHastingsRandomWalkSampler()` from the module `littleballoffur`. The intuition behind the method is to initialise a random subgraph $S$ of our full network $G$ from which we iteratively remove and add nodes in order to mimic some topological properties of $G$, in this case the degree-distriubtion. This is of course not without consequences as we remove of the complexity in our network and thereby lose information, however we deem this nesecarry to reduce our network to a more manageble size.

In [175]:
# We create a dictionary with the index each pagename
index_dict = {i:k for k, i in enumerate(df['title'])}

# We create an edgelist 
edge_list = []
for node, edges in zip(df['title'].tolist(), df['edges'].tolist()):
    for edge in edges:
        edge_list.append((index_dict[node], index_dict[edge]))
edge_list = [e for e in edge_list if e[0] != e[1]] #  ... and remove self-loops

# We model the acutal graph and extract the gcc
G = nx.Graph()
G.add_edges_from(edge_list)
gcc = max(nx.connected_components(G), key=len)
G = G.subgraph(gcc)

# littleballoffur requires all names to be translated into integers
G = nx.relabel.convert_node_labels_to_integers(G)

# We make a sample with 0.25 of the gcc's observations
number_of_nodes = int(0.25 * G.number_of_nodes())
sampler = MetropolisHastingsRandomWalkSampler(number_of_nodes = number_of_nodes)
sampled_supgraph = sampler.sample(G)

To asses the bias induced by our sampling we compare the networks in terms of their degree exponent and transitivity. This way we can make sure that the networks degree distribution and level of clusterness is somewhat comparable. 

In [176]:
%%capture
# Calculating the exponent
G_degree_dist = [i[1] for i in G.degree]
sampled_supgraph_degree_dist = [i[1] for i in sampled_supgraph.degree]
G_exponent = powerlaw.Fit(G_degree_dist)
sampled_supgraph_exponent = powerlaw.Fit(sampled_supgraph_degree_dist)

# Calculating the transitivity
G_transitivity = nx.transitivity(G)
sampled_supgraph_transitivity = nx.transitivity(sampled_supgraph)

In [177]:
print(f"Degree distribution exponent full network G: {G_exponent.power_law.alpha}")
print(f"Degree distribution exponent supgraph S: {sampled_supgraph_exponent.power_law.alpha}")
print("-"*70)
print('Transitivity full network G: {:.4f}'.format(transitivity))
print('Transitivity supgraph S: {:.4f}'.format(transitivity_sampled))

Degree distribution exponent full network G: 3.2974527129230946
Degree distribution exponent supgraph S: 3.8342121796821265
----------------------------------------------------------------------
Transitivity full network G: 0.1929
Transitivity supgraph S: 0.2991


Reassuringly, the exponent of the two networks are fairly similiar suggesting that the degree distributions follows the somewhat same structure. A bit more worrying is the difference in transitivity that shows that we have oversestimated the transitivity in our supgraph. The result of this is that our network are more dense with higher number of "triads", than what really is the case. Nontheless we still decide to use the supgraph for our further analysis. We can now model our final network and corresponding DataFrame that we will use for our analysis based on the nodes from the supgraph. Finally we restrict this network to the weakly connected component as we create a *directed* network. 

In [181]:
# We define a list of nodes to keep and remove edges that refers to nodes that we have removed
nodes_to_keep = [list(index_dict.keys())[i] for i in list(sampled_supgraph.nodes())]
df = df[df['title'].isin(nodes_to_keep)].reset_index()[["title", "parent", "depth", "text", "edges"]]
df = remove_edges_not_in_nodelist(df)

# Set attributes
node_attr = df[["title", "parent", "depth"]].set_index("title").to_dict(orient='index')

# Create an edgelist
edge_list = []
for node, edges in zip(df['title'].tolist(), df['edges'].tolist()):
    for edge in edges:
        edge_list.append((node, edge))

# Model the network
G = nx.DiGraph()
G.add_edges_from(edge_list)
nx.set_node_attributes(G, node_attr)

# Extract the weakly connected component 
gcc = max(nx.weakly_connected_components(G), key=len)
G = G.subgraph(gcc)

# Add a gcc column to our final DataFrame
df["gcc"] = df["title"].apply(lambda x: 1 if x in G.nodes() else 0)

  0%|          | 0/4302 [00:00<?, ?it/s]

In [189]:
# Saving the edgelist as the a directed network can not be pickled
with open("Final_edge_list.pickle", 'wb') as f:
    pickle.dump(edge_list, f)
    
# Saving the node attributes
with open("Final_node_attr.pickle", 'wb') as f:
    pickle.dump(node_attr, f)

# Add preprocessing

In this section we conduct light preprocessing since the wikipedia documents are fairly clean. We do the following preprocessing steps:

- **1 Getting the clean text from the Wikipedia pages:**
The Wikipedia pages are structued in a page-specific text section and a section with links called "See Also". We are only intereseted in first one. We clean the text further by removing all non-alphanumerical characters.

- **2 Remove stopwords:** 
To reduce the size of the pages vocabulary we remove all stopwords as they do not carry a lot of information relative to their size. 

- **3 Lemmatization of words:**
Lemmatization refer to the reduction of words to its syntactical root to align words despite grammatical modifications which is useful for our analysis of text. We therefore add a column to our final DataFrame with lemmatized words.

- **4 Tokenisation of words:**
We split up all the lemmatized pages into list of words (or tokens) and add them to a new column in our final DataFrame.

In [190]:
def clean_text(text: str) -> str:
    """
    Extracts clean text from the Wikipedia-pages by splitting
    non-alphanumerical characters on the "See also" section and removing
    """
    text = text.split('See also')[0]
    text = re.sub('\W+', ' ', text)
    return text.lower()

def remove_stopwords(text: str) -> str:
    """
    Removes nltk stopwords bounded by whitespace and replace it
    with whitespace.
    """
    patterns = set(stopwords.words('english'))
    for pattern in patterns:
        if re.search(' '+pattern+' ', text):          
            text = re.sub(' '+pattern+' ', ' ', text) 
    return text

def lemmatize(text: str) -> str:
    """
    Lemmatize all text.
    """
    lemmatizer = WordNetLemmatizer()
    text = word_tokenize(text) 
    sent_lemmatized = [lemmatizer.lemmatize(word) for word in text]
    return ' '.join(sent_lemmatized)

def word_tokenize(text: str) -> str:
    """
    Tokenize all text.
    """
    text = WordPunctTokenizer().tokenize(text)
    return text

# we apply all functions to the text
warnings.filterwarnings('ignore')  # Ignore DepreciationError
tqdm.pandas()
df['cleaned_text'] = df['text'].astype(str).progress_apply(lambda x: clean_text(x))
df['lemmatized'] = df['cleaned_text'].astype(str).progress_apply(lambda x: remove_stopwords(x))
df['lemmatized'] = df['lemmatized'].astype(str).progress_apply(lambda x: lemmatize(x))
df['tokens'] = df['lemmatized'].astype(str).progress_apply(lambda x: word_tokenize(x))

  0%|          | 0/4302 [00:00<?, ?it/s]

  0%|          | 0/4302 [00:00<?, ?it/s]

  0%|          | 0/4302 [00:00<?, ?it/s]

  0%|          | 0/4302 [00:00<?, ?it/s]

In [192]:
# Save final DataFrame
df.to_pickle("Final_df.pickle")

In [200]:
df.groupby("parent").count()

,title,depth,text,edges,gcc,cleaned_text,lemmatized,tokens
parent,,,,,,,,
anthropology,510,510,510,510,510,510,510,510
economics,1020,1020,1020,1020,1020,1020,1020,1020
political_science,1614,1614,1614,1614,1614,1614,1614,1614
psychology,543,543,543,543,543,543,543,543
sociology,615,615,615,615,615,615,615,615
